# Production mix - Model 11

## Situation

You own a boutique pottery business, making and selling two types of large ornamental products called Lunar Orb and Solar Disc. Given constraints on staff hours, available materials, and product sales, your objective is to maximize the total profit margin from the shop.

## Implementation

Linear Program (LP), using SciPy. The data is loaded from an external json file.

Note: In Jupyter Lab a json file is opened, by default, in json format. To make changes to the file, open the file with the Editor (right-click > Open with > Editor).

## Source

Replicates an Excel model described in article "Production mix via graphical LP" at https://www.solvermax.com/blog/production-mix.

In [1]:
# Import dependencies

from scipy.optimize import linprog
import pandas as pd
import numpy as np
import os.path
import json

In [2]:
# Get data

DataFilename = os.path.join('.', 'productiondata11.json')
with open(DataFilename, 'r') as f:
    Data = json.load(f)

In [3]:
# Declarations

Name = Data['Name']
Hours = Data['Hours']
kg = Data['kg']
SalesLimit = Data['SalesLimit']
VarInitial = Data['VarInitial']   # Not used
VarLBounds = Data['VarLBounds']
VarUBounds = Data['VarUBounds']
Engine = Data['Engine']
TimeLimit = Data['TimeLimit']

Coefficients = Data['Coefficients']
Products = list(Coefficients.keys())
NumProducts = len(Products)

In [4]:
# Define model

Margin = np.zeros(NumProducts)
People = np.zeros(NumProducts)
Materials = np.zeros(NumProducts)
Sales = np.zeros(NumProducts)
for p in Products:
    i = int(p)-1
    Margin[i]    = -Coefficients[p]['Margin']  # Need to negate, as SciPy always minimizes, but we want to maximize
    People[i]    =  Coefficients[p]['People']
    Materials[i] =  Coefficients[p]['Materials']
    Sales[i]     =  Coefficients[p]['Sales']
    
ObJCoeff = Margin
Constraints = [People, Materials, Sales]
rhs = [Hours, kg, SalesLimit];

In [5]:
# Solve model

Model = linprog(c = ObJCoeff, A_ub = Constraints, b_ub = rhs, bounds = [(VarLBounds, VarUBounds)], method = Engine, options = {'time_limit': TimeLimit})

In [6]:
# Process results

WriteSolution = False
Optimal = False
Condition = Model.success

if Condition:
    Optimal = True
    WriteSolution = True

In [7]:
# Write output

print(Name, '\n')
print('Status: ', Model.success, '\n')

if WriteSolution:
    print(f"Total margin = ${-Model.fun:,.2f}\n")  # Need to negate, as we're maximizing
    pd.options.display.float_format = "{:,.4f}".format
    ProductResults = pd.DataFrame()
    for p in Products:
        ProductResults.loc[p, 'Production'] = Model.x[int(p)-1]
    display(ProductResults)
    ConstraintStatus = pd.DataFrame(columns=['Slack', 'Dual'])
    for c in range(len(Constraints)):
        ConstraintStatus.loc[c] = [Model.slack[c], Model['ineqlin']['marginals'][c]]
    display(ConstraintStatus)
else:
    print('No solution loaded\n')
    print('Model:')
    print(Model)

Boutique pottery shop - Model 11 

Status:  True 

Total margin = $3,076.92



,Production
1,6.4103
2,12.8205


,Slack,Dual
0,41.6667,-0.0000
1,0.0000,-6.1538
2,0.0000,-15.3846
